In [1]:
import sys 
import os
import pandas as pd
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from utils.utils import save_experiment, train_and_evaluate_non_linear_svm, evaluate_model_on_parquet, find_best_trained_monel
from configs.config_local import DATASET_PATH, ITW_DATASET_PATH, MODELS_PATH
import joblib


In [2]:
train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

# Assuming train_data_path and val_data_path are parquet files
C_values = [0.1, 1, 10]
gamma_values = [0.001, 0.01, "scale"]

best_score = 0
best_model = None
best_params = None

for C in C_values:
    for gamma in gamma_values:
        svc_params = {
            "kernel": 'rbf',
            "C": C,
            "gamma": gamma,
            "class_weight": "balanced",
            "max_iter": 20000,
        }

        pipeline, metrics, svc_params_out, feature_names, metadata_extra = train_and_evaluate_non_linear_svm(
            train_data_path, test_data_path, svc_params
        )
        save_experiment(
            model=pipeline,
            metrics=metrics,
            experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "rbf_svm"),
            model_params=svc_params,
            feature_names=feature_names,
            metadata_extra=metadata_extra,
        )


        # Using ROC AUC to pick best
        if metrics["roc_auc"] > best_score:
            best_score = metrics["roc_auc"]
            best_model = pipeline
            best_params = svc_params_out

print("Best hyperparameters:", best_params)
print("Validation ROC AUC:", best_score)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.984039   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.599396   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -309.998932   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -329.094482   
4  real  file1001.wav_16k.wav_norm.wav_mono.wav_silence...  -369.833069   

   mfcc_std_0  mfcc_mean_1  mfcc_std_1  mfcc_mean_2  mfcc_std_2  mfcc_mean_3  \
0  106.660751   134.430344   62.437195     9.798609   39.744644    13.036348   
1  115.704865   106.857346   50.405762    12.835211   42.213852    37.456020   
2   72.639381   143.500931   32.701843    16.544865   31.033148    49.439816   
3  119.318130    90.687096   70.433701    12.409759   38.385540    40.940025   
4  129.029114    77.314659   67.398788     2.168967   48.882267    34.795979   

   mfcc_std_3  ...  mel_spectrogram_mean_123  mel_spectrogram_std_12

c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_230718
Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_231224
Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_231538


c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_232543
Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_233106
Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_233422


c:\Users\konst\anaconda3\envs\audio_deepfake_py313\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_234410
Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\rbf_svm\exp_20260117_234951
Best hyperparameters: {'kernel': 'rbf', 'C': 10, 'gamma': 'scale', 'class_weight': 'balanced', 'max_iter': 20000}
Validation ROC AUC: 0.9769799913524475


Test Best Model to ITW

In [ ]:
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
metrics, metadata_extra = evaluate_model_on_parquet(best_model, itw_test_data_path)
print(metrics)

{'accuracy': 0.6232578812025779, 'precision': 0.4947026246087166, 'recall': 0.34884115799303844, 'f1': 0.5663124088573276}


In [4]:
itw_test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
rbf_models_path = os.path.join(MODELS_PATH,'rbf_svm')
exp = find_best_trained_monel(rbf_models_path,'f1')
best_f1_model = joblib.load(os.path.join(MODELS_PATH,'rbf_svm',exp,'model.joblib'))


metrics, metadata_extra = evaluate_model_on_parquet(best_f1_model, itw_test_data_path)
print(metrics)

Best model according to f1: exp_20260117_234410, f1=0.8467561929764884
{'accuracy': 0.5555414457601828, 'precision': 0.4398461621797302, 'recall': 0.6893624246540453, 'f1': 0.5548302591620256}
